## Das erste Model

In [1]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)/'images' # Lädt ein Datenset mit Katzen- und Hundebildern

def is_cat(x): return x[0].isupper()

dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)


/opt/miniconda3/envs/fastai/lib/python3.10/site-packages/fastai/vision/learner.py:303: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /Users/lukelotshakulongo/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████████████████████████████████| 83.3M/83.3M [00:07<00:00, 12.4MB/s]


epoch,train_loss,valid_loss,error_rate,time
0,0.185122,0.014785,0.008119,01:09


epoch,train_loss,valid_loss,error_rate,time
0,0.047888,0.017226,0.007442,01:31


### Objektposition erkennen: Segmentierung
Jedes Pixel im Bild einer Klasse zuordnen (z. B. Baum, Straße, Auto)


In [2]:
from fastai.vision.all import *
path = untar_data(URLs.CAMVID_TINY)

dls = SegmentationDataLoaders.from_label_func(
    path, bs=8, fnames=get_image_files(path/"images"),
    label_func=lambda o: path/'labels'/f'{o.stem}_P{o.suffix}',
    codes=np.loadtxt(path/'codes.txt', dtype=str))

learn = unet_learner(dls, resnet34)
learn.fine_tune(8)


epoch,train_loss,valid_loss,time
0,2.706262,2.073185,00:07


epoch,train_loss,valid_loss,time
0,1.622544,1.368484,00:03
1,1.449582,1.440078,00:02
2,1.367531,1.018627,00:02
3,1.231648,0.859655,00:02
4,1.103282,0.810725,00:02
5,0.998844,0.722900,00:02
6,0.910162,0.713778,00:02
7,0.842216,0.706034,00:02


###  Texte verstehen: Sentimentanalyse (NLP)
Ziel: Verstehen, ob ein Filmreview positiv oder negativ ist 


In [ ]:
from fastai.text.all import *

dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(4, 1e-2)

learn.predict("I really liked that movie!")
# Ausgabe: ('pos', tensor(1), tensor([0.0041, 0.9959]))


In [6]:
doc(learn.predict) # Öffnet Hilfe + Link zur Dokumentation und sogar den Quellcode

python(1794) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


## Deep Learning mit Tabellen: Tabular Data
Vorhersage auf Basis von Spalten in einer Tabelle (z. B. Einkommen basierend auf Alter, Bildung usw.)

In [7]:
from fastai.tabular.all import *
path = untar_data(URLs.ADULT_SAMPLE)

dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(3)

/opt/miniconda3/envs/fastai/lib/python3.10/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


epoch,train_loss,valid_loss,accuracy,time
0,0.379801,0.381091,0.825092,00:09
1,0.348716,0.368543,0.831235,00:04
2,0.350568,0.366212,0.828624,00:04


In [8]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,8.0,16.0,3.0,6.0,1.0,5.0,1.0,1.061339,-0.345255,-0.026613,0.0,0.0
1,5.0,10.0,3.0,5.0,1.0,5.0,1.0,0.252888,-0.207401,1.148387,1.0,1.0
2,5.0,12.0,3.0,15.0,1.0,5.0,1.0,1.649304,-0.251821,-0.418280,0.0,0.0
3,7.0,12.0,6.0,4.0,5.0,5.0,1.0,-0.408572,1.619538,-0.418280,0.0,0.0
4,5.0,8.0,3.0,11.0,1.0,5.0,1.0,-0.482068,-0.822734,0.756720,0.0,0.0
5,5.0,10.0,3.0,5.0,1.0,5.0,1.0,-0.482068,-0.813284,1.148387,1.0,1.0
6,5.0,12.0,5.0,7.0,4.0,5.0,1.0,-0.776050,-0.446866,-0.418280,0.0,0.0
7,5.0,16.0,7.0,2.0,2.0,5.0,1.0,0.546870,-0.767647,-0.026613,0.0,0.0
8,5.0,16.0,5.0,9.0,4.0,5.0,1.0,-0.849545,-0.022145,-0.026613,0.0,0.0


## Empfehlungssysteme: Was könnte dir gefallen?
Ziel: Vorhersage, welche Filme ein Nutzer mögen könnte

In [9]:
from fastai.collab import *
path = untar_data(URLs.ML_SAMPLE)

dls = CollabDataLoaders.from_csv(path/'ratings.csv')
learn = collab_learner(dls, y_range=(0.5,5.5))

learn.fine_tune(10)
learn.show_results()

epoch,train_loss,valid_loss,time
0,1.491501,1.455243,00:01


epoch,train_loss,valid_loss,time
0,1.375454,1.405861,00:00
1,1.275629,1.257891,00:00
2,1.046942,0.969511,00:00
3,0.823577,0.787967,00:00
4,0.709716,0.740930,00:00
5,0.635881,0.717075,00:00
6,0.615767,0.708287,00:00
7,0.597545,0.703666,00:00
8,0.592504,0.702671,00:00
9,0.596240,0.702510,00:00


,userId,movieId,rating,rating_pred
0,13.0,50.0,4.0,4.010127
1,26.0,69.0,5.0,4.203037
2,54.0,33.0,4.0,3.231553
3,1.0,37.0,4.0,3.357182
4,8.0,23.0,2.0,2.992038
5,91.0,4.0,5.0,3.186765
6,17.0,6.0,5.0,4.262587
7,92.0,63.0,4.0,4.440350
8,39.0,82.0,3.5,3.855596
